## Bibliotecas

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm  #regressão linear
from statsmodels.tsa.stattools import coint, adfuller #pra verificar cointegração por engle-granger e dickey-fuller
import itertools
from datetime import datetime, timedelta

## Funções

In [46]:
# Carregar dados
def carregar_dados(moeda1, moeda2, PATH="5m"):
    path_ativo_1 = f'../../data/fechamentos/{moeda1}USDT_{PATH}_data.csv'
    path_ativo_2 = f'../../data/fechamentos/{moeda2}USDT_{PATH}_data.csv'

    ativo1 = pd.read_csv(path_ativo_1, parse_dates=['timestamp'], index_col='timestamp')
    ativo2 = pd.read_csv(path_ativo_2, parse_dates=['timestamp'], index_col='timestamp')

    # Elimina NaNs restantes
    df = pd.DataFrame({
        'Ativo1': ativo1['close'],
        'Ativo2': ativo2['close']
    }).dropna()

    return df



# Teste de cointegração
def testar_cointegracao(series1, series2, signif=0.05):
    """
    Retorna True se as séries forem cointegradas segundo:
    - ADF individual: ambas não estacionárias
    - Engle-Granger: resíduos estacionários
    """
    # Teste Dickey-Fuller individual
    adf1_p = adfuller(series1)[1]
    adf2_p = adfuller(series2)[1]
    
    # Se alguma série for estacionária, não faz sentido testar cointegração
    if adf1_p <= signif or adf2_p <= signif:
        return False
    
    # Teste Engle-Granger (Cointegração)
    _, p_coint, _ = coint(series1, series2)
    
    return p_coint, p_coint < signif

from statsmodels.tsa.stattools import coint, adfuller

def testar_cointegracao_movel(series1, series2, janela, signif=0.05):
    """
    Testa cointegração móvel entre duas séries de preços usando Engle-Granger.
    
    Passos:
    1. Para cada janela de tamanho 'janela', faz regressão linear:
        series1 = alpha + beta * series2 + residuos
    2. Testa Engle-Granger (coint) na janela.
    3. Testa Dickey-Fuller (ADF) nos resíduos da regressão.
    
    Retorna True para cada janela se:
        - pvalor Engle-Granger < signif
        - pvalor ADF nos resíduos < signif
    Caso contrário, retorna False.
    """

    resultados = []

    #Janela válida até final da série
    for i in range(janela, len(series1)):

        # Pega apenas a janela atual (e.g: 5 até 10 se: janela=5 e i=10)
        window1 = series1[i - janela:i]
        window2 = series2[i - janela:i]

        # Regressão linear: series1 ~ series2
        X = sm.add_constant(window2)  # adiciona intercepto
        model = sm.OLS(window1, X).fit()
        residuos = model.resid

        # Teste Engle-Granger
        _, pval_coint, _ = coint(window1, window2)

        # Teste Dickey-Fuller nos resíduos
        _, pval_adf, _, _, _, _ = adfuller(residuos)

        # Só é cointegrado se ambos testes rejeitarem H0
        cointegrado = (pval_coint < signif) and (pval_adf < signif)
        resultados.append(cointegrado)

    print(resultados)

    return resultados
# Zscore
def calcular_zscore(spread, window=60):
    rolling_mean = spread.rolling(window=window).mean() #média da última janela
    rolling_std = spread.rolling(window=window).std() #desvio padrão da última janela

    # Substituir stds muito baixos por um mínimo
    epsilon = 1e-8
    rolling_std = rolling_std.replace(0, epsilon).fillna(epsilon)

    zscore = (spread - rolling_mean) / rolling_std  #zscore = (x- média_janela)/(desvio_padrao_janela)

    return rolling_mean, rolling_std, zscore #média da janela, desvio padrao, zscore

# Zscore
def calcular_media_ativos(ativo1, ativo2, window=60):
    ativo1_mean = ativo1.rolling(window=window).mean() #média da última janela
    ativo2_mean = ativo2.rolling(window=window).mean() #média da última janela

    ativo1_std = ativo1.rolling(window=window).std() #std da media movel
    ativo2_std = ativo2.rolling(window=window).std() #std da media movel


    return ativo1_mean, ativo2_mean, ativo1_std, ativo2_std  #média da janela, desvio padrao, zscore


#spread
def calcular_spread(serie1, serie2):
    serie1 = pd.to_numeric(serie1, errors='coerce')
    serie2 = pd.to_numeric(serie2, errors='coerce')
    spread = serie1 - serie2

    return spread

def vale_a_pena_operar(cotacao_atual, cotacao_futura_esperada, taxa=0.01, tipo_operacao='compra'):
    """
    Verifica se vale a pena operar o ativo (comprado ou vendido), considerando as taxas.

    Parâmetros:
    - cotacao_atual: preço atual do ativo
    - cotacao_futura_esperada: preço esperado na venda (se comprado) ou recompra (se vendido)
    - taxa: taxa cobrada em cada operação (compra ou venda), como 0.01 para 1%
    - tipo_operacao: 'compra' ou 'venda' (representa a posição inicial)

    Retorna:
    - True se a operação vale a pena (lucro cobre as taxas), False caso contrário
    """

    if tipo_operacao == 'compra':
        custo_total = cotacao_atual * (1 + taxa)
        valor_recebido = cotacao_futura_esperada * (1 - taxa)
        return valor_recebido > custo_total
    
    elif tipo_operacao == 'venda':
        receita_liquida = cotacao_atual * (1 - taxa)
        custo_recompra = cotacao_futura_esperada * (1 + taxa)
        return receita_liquida > custo_recompra
    
    else:
        raise ValueError("tipo_operacao deve ser 'compra' ou 'venda'")


# Estratégia de sinalização
def gerar_sinais_com_stoploss(df, zscore_compra_e_venda, zscore_encerrar_posicao, stop_loss, cooldown_stop_loss, janela, taxa=0.001):
    series1 = df['Ativo1']
    series2 = df['Ativo2']

    limite_superior =zscore_compra_e_venda
    limite_inferior = -zscore_compra_e_venda

    spread = calcular_spread(series1, series2)
    rolling_mean, rolling_std, zscore = calcular_zscore(spread, janela) #Zscore com janela movel
    media_movel_ativo1, media_movel_ativo2, ativo1_std, ativo2_std = calcular_media_ativos(series1, series2, janela) #media movel do ativo 1 e 2

    # Calcular ATR do spread
    spread_diff = spread.diff().abs()
    atr = spread_diff.rolling(window=janela).mean().bfill()

    nomes_posicoes = ["neutro", "compra_1_vende_2", "vende_1_compra_2"]
    posicao = nomes_posicoes[0] #comprado_vendido, vendido_comprado, encerrar, Neutro
    cooldown = 0
    sinais_compra_e_venda = []
    preco_entrada = None

    for i in range(len(df)):
        spread_atual = spread.iloc[i]
        atr_atual = atr.iloc[i]

        # Se em cooldown, não faz nada
        if cooldown > 0:
            cooldown -= 1
            sinais_compra_e_venda.append(nomes_posicoes[0])
            posicao = nomes_posicoes[0]
            preco_entrada = None
            continue

        # Verifica stop loss baseado em ATR
        if posicao != nomes_posicoes[0] and preco_entrada is not None:
            prejuizo = (
                spread_atual - preco_entrada if posicao == nomes_posicoes[2]
                else preco_entrada - spread_atual
            )
            if prejuizo > stop_loss * atr_atual:
                posicao = nomes_posicoes[0]
                cooldown = cooldown_stop_loss*janela
                preco_entrada = None
                sinais_compra_e_venda.append("neutro")
                continue  # sai daqui e não avalia entrada nova

        # Só tenta entrar em nova posição se está neutro e não em cooldown
        if posicao == nomes_posicoes[0]:
            if zscore.iloc[i] > limite_superior and vale_a_pena_operar(df['Ativo1'].iloc[i], media_movel_ativo1.iloc[i], taxa, 'venda'):
                posicao = nomes_posicoes[2] #entra vendido no ativo 1 e comprado no 2
                preco_entrada = spread_atual
            if zscore.iloc[i] < limite_inferior and vale_a_pena_operar(df['Ativo1'].iloc[i], media_movel_ativo1.iloc[i], taxa, 'compra'):
                posicao = nomes_posicoes[1] #entra comprado no ativo 1 e vendido no 2
                preco_entrada = spread_atual

        # Verifica se deve encerrar a posição com base no z-score
        if (-0.5 < zscore.iloc[i] < zscore_encerrar_posicao):
            posicao = nomes_posicoes[0] #posicao mantem neutra ou encerra posicao anterior
            preco_entrada = None
            
        sinais_compra_e_venda.append(posicao)
    
    # Resultado final
    df_sinais = pd.DataFrame({
        'timestamp': df.index,
        'Ativo1': pd.to_numeric(series1, errors='coerce'),
        'Ativo2': pd.to_numeric(series2, errors='coerce'),
        'ativo1_mean': media_movel_ativo1,
        'ativo2_mean': media_movel_ativo2,
        'ativo1_std': ativo1_std, 
        'ativo2_std': ativo2_std,
        'spread': pd.to_numeric(spread, errors='coerce'),
        'rolling_mean': rolling_mean.values,
        'rolling_std': rolling_std.values,
        'zscore': zscore.values,
        'sinal': sinais_compra_e_venda
    }).dropna()
    
    df_sinais.to_excel('resultados/estrategia.xlsx', index=False)

    return df_sinais

# Estratégia de sinalização
def gerar_sinais(df, zscore_compra_e_venda, zscore_encerrar_posicao, stop_loss, cooldown_stop_loss, janela, taxa=0.001):
    series1 = df['Ativo1']
    series2 = df['Ativo2']

    limite_superior =zscore_compra_e_venda
    limite_inferior = -zscore_compra_e_venda

    # Teste de cointegração móvel
    resultados_cointegracao = testar_cointegracao_movel(series1, series2, janela)
    # Preenche início com False para alinhar ao tamanho da série
    status_cointegracao = [False]*janela + resultados_cointegracao
    status_cointegracao = pd.Series(status_cointegracao, index=series1.index)
    
    spread = calcular_spread(series1, series2)
    rolling_mean, rolling_std, zscore = calcular_zscore(spread, janela) #Zscore com janela movel
    media_movel_ativo1, media_movel_ativo2, ativo1_std, ativo2_std = calcular_media_ativos(series1, series2, janela) #media movel do ativo 1 e 2

    nomes_posicoes = ["neutro", "compra_1_vende_2", "vende_1_compra_2"]
    posicao = nomes_posicoes[0] #comprado_vendido, vendido_comprado, encerrar, Neutro

    
    sinais_compra_e_venda = []

    for i in range(len(df)):
        linha_df = df.iloc[i]
        '''
        if zscore.iloc[i] > limite_superior:
            posicao = nomes_posicoes[2] #entra vendido no ativo 1 e comprado no 2
        if zscore.iloc[i] < -limite_inferior:
            posicao = nomes_posicoes[1] #entra comprado no ativo 1 e vendido no 2
        '''
        if zscore.iloc[i] > limite_superior and vale_a_pena_operar(df['Ativo1'].iloc[i], media_movel_ativo1.iloc[i], taxa, 'venda'):
            posicao = nomes_posicoes[2] #entra vendido no ativo 1 e comprado no 2
        if zscore.iloc[i] < limite_inferior and vale_a_pena_operar(df['Ativo1'].iloc[i], media_movel_ativo1.iloc[i], taxa, 'compra'):
            posicao = nomes_posicoes[1] #entra comprado no ativo 1 e vendido no 2


        if (-zscore_encerrar_posicao < zscore.iloc[i] < zscore_encerrar_posicao):
            posicao = nomes_posicoes[0] #posicao mantem neutra ou encerra posicao anterior
        ##if stoploss
        sinais_compra_e_venda.append(posicao)
    
    # Resultado final
    df_sinais = pd.DataFrame({
        'timestamp': df.index,
        'status_cointegracao': status_cointegracao,
        'Ativo1': pd.to_numeric(series1, errors='coerce'),
        'Ativo2': pd.to_numeric(series2, errors='coerce'),
        'ativo1_mean': media_movel_ativo1,
        'ativo2_mean': media_movel_ativo2,
        'ativo1_std': ativo1_std, 
        'ativo2_std': ativo2_std,
        'spread': pd.to_numeric(spread, errors='coerce'),
        'rolling_mean': rolling_mean.values,
        'rolling_std': rolling_std.values,
        'zscore': zscore.values,
        'sinal': sinais_compra_e_venda
    }).dropna()
    


    df_sinais.to_excel('resultados/estrategia.xlsx', index=False)

    return df_sinais

def simular_retorno_por_trade(df, capital_inicial=10000, taxa=0.001):
    """
    Simula retorno por trade de uma estratégia de pares.
    
    Parâmetros:
    df: DataFrame com colunas ['Ativo1', 'Ativo2', 'sinal']
         - sinal: 'compra_1_vende_2', 'vende_1_compra_2' ou 'neutro'
    capital_inicial: capital inicial para simulação
    taxa: custo proporcional por operação (ex: 0.001 = 0.1%)
    
    Retorna:
    retorno_pct_total: O retorno percentual ao fim daquela simulação
    df_resultado: DataFrame com capital acumulado e retorno por trade
    """

    series1 = df['Ativo1']
    series2 = df['Ativo2']
    sinais = df['sinal']

    capital = capital_inicial
    capital_series = [capital]
    retornos_trade = [None]

    posicao = None
    entrada_indice = None

    for i in range(1, len(sinais)):
        ret = None

        # Entrada na operação
        if posicao is None and sinais.iloc[i] != 'neutro':
            posicao = sinais.iloc[i]
            entrada_indice = i

        # Saída da operação
        elif posicao is not None and sinais.iloc[i] == 'neutro':
            preco_entrada_1 = series1.iloc[entrada_indice]
            preco_entrada_2 = series2.iloc[entrada_indice]

            preco_saida_1 = series1.iloc[i]
            preco_saida_2 = series2.iloc[i]

            # Retorno bruto de cada perna
            retorno_1 = (preco_saida_1 / preco_entrada_1) - 1
            retorno_2 = (preco_saida_2 / preco_entrada_2) - 1

            # Desconta custos: taxa de entrada + taxa de saída
            retorno_1 -= 2 * taxa
            retorno_2 -= 2 * taxa

            # Retorno líquido da operação combinada
            if posicao == 'compra_1_vende_2':
                ret = retorno_1 - retorno_2
            elif posicao == 'vende_1_compra_2':
                ret = retorno_2 - retorno_1

            # Capital ajustado pelo retorno da operação
            capital *= (1 + ret)
            posicao = None
            entrada_indice = None

        capital_series.append(capital)
        retornos_trade.append(ret)

    retorno_pct_total = (capital / capital_inicial) - 1

    df_resultado = df.copy()
    df_resultado['capital'] = capital_series
    df_resultado['retorno_trade'] = retornos_trade

    print(f"Retorno total: {retorno_pct_total:.2%}")
    return retorno_pct_total, df_resultado

def simular_estrategia(moeda1, moeda2, zscore_compra_e_venda, zscore_encerrar_posicao, 
                       stop_loss, cooldown_stop_loss, janela, data_inicial, data_final, 
                       periodo_observacoes="1d", taxa=0.001, capital_inicial=10000):
    
    # Carregar dados
    df = carregar_dados(moeda1, moeda2, periodo_observacoes).loc[data_inicial:data_final]


    # Testar cointegração
    '''
    p_valor = testar_cointegracao(df['Ativo1'], df['Ativo2'])
    if p_valor > 0.05:
        print(f"As séries {moeda1}-{moeda2} não são cointegradas (p-valor = {p_valor:.4f})")
        return None, None, None
    '''

    # Gerar sinais
    df_sinais = gerar_sinais(df, zscore_compra_e_venda, zscore_encerrar_posicao, stop_loss, cooldown_stop_loss, janela, taxa)

    # Simular retorno
    retorno_pct, df_resultado = simular_retorno_por_trade(df_sinais, capital_inicial, taxa)

    # Calcular retornos percentuais por trade
    retornos_trade = df_resultado['retorno_trade'].dropna()

    # Calcular métricas
    sharpe = calcular_sharpe(retornos_trade)
    retorno_risco = retorno_ajustado_ao_risco(retornos_trade)

    return retorno_pct, retorno_risco, sharpe

def retorno_ajustado_ao_risco(serie_retorno):
    serie_retorno = pd.Series(serie_retorno).dropna()
    if len(serie_retorno) == 0:
        return 0
    retorno_total = (1 + serie_retorno).prod() - 1
    risco = serie_retorno.std()
    if risco == 0:
        return 0
    return retorno_total / risco

def calcular_sharpe(serie_retorno, taxa_livre_risco=0.0):
    serie_retorno = pd.Series(serie_retorno).dropna()
    if len(serie_retorno) == 0:
        return 0
    media_excesso_retorno = serie_retorno.mean() - taxa_livre_risco
    desvio = serie_retorno.std()
    if desvio == 0:
        return 0
    sharpe = media_excesso_retorno / desvio
    return sharpe
    

In [17]:
# Intervalo para teste de cointegração
data_inicial = '2025-01-01'
data_final   = '2025-07-28'


# Converter string para objeto datetime
data = datetime.strptime(data_inicial, "%Y-%m-%d")
nova_data = data - timedelta(days=90)
nova_data.strftime("%Y-%m-%d")
print(nova_data)

2024-10-03 00:00:00


## Teste da estratégia

In [47]:
# Algoritmo de otimizacao
zscv = 2.5
zse = 0.5

stop_loss = 1.5
cooldown = 0

janela = 11

lista_moedas = ['BTC', 'WBTC', 'ETH', 'ADA', 'BNB', 'SOL', 'XRP']
tempo_obs =["5m","1h", "1d"]
dfcodigo = carregar_dados("LTC", "TRX", tempo_obs[2])

# Intervalo para teste de cointegração
data_inicial = '2025-01-01'
data_final   = '2025-07-28'
df_filtrado = dfcodigo.loc[data_inicial:data_final]


df_sinalizado = gerar_sinais(df_filtrado, zscv, zse, stop_loss, cooldown, janela, 0.001) #gera os sinais de compra_venda_encerramento
#df_sinalizado = gerar_sinais_com_stoploss(df_filtrado, zscv, zse, stop_loss, cooldown, janela, 0.01) #gera os sinais de compra_venda_encerramento

retorno_pct, df_com_retorno_financeiro = simular_retorno_por_trade(df_sinalizado) #simula os retornos

#Valores pra gráfico
df_com_retorno_financeiro['ativo1_desvio_preco'] =  df_com_retorno_financeiro['Ativo1'] - df_com_retorno_financeiro['ativo1_mean']
df_com_retorno_financeiro['ativo1_std_preco'] = df_com_retorno_financeiro['ativo1_desvio_preco'] / df_com_retorno_financeiro['ativo1_std']

df_com_retorno_financeiro['ativo2_desvio_preco'] = df_com_retorno_financeiro['Ativo2'] - df_com_retorno_financeiro['ativo2_mean'] 
df_com_retorno_financeiro['ativo2_std_preco'] = df_com_retorno_financeiro['ativo2_desvio_preco'] / df_com_retorno_financeiro['ativo2_std']



df_com_retorno_financeiro.to_excel('resultados/retornos.xlsx', index=False)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, False, True, False, False, False, False, False, False, False, False, True, False, False, True, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, False, False, False, False, False, False, 

In [9]:
retorno_pct, retorno_risco, sharpe = simular_estrategia(
    moeda1='BTC',
    moeda2='BNB',
    zscore_compra_e_venda=2,
    zscore_encerrar_posicao=0.5,
    stop_loss=0,
    cooldown_stop_loss=0,
    janela=11,
    data_inicial='2025-06-20',
    data_final='2025-07-19',
    periodo_observacoes="1d",
)

#print(f"Retorno percentual Simulado: {retorno_pct*100:.2f}%")

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False]
Retorno total: 4.78%


## Testar pares de cointegrados

In [42]:
moedas = ['BTC', 'BNB', 'ETH', 'SOL', 'ADA', 'XRP', 'DOGE', 'LTC', 'TRX', 'DOT', 'SHIB', 'BCH', 'TON', 'DAI', 'AVAX']

caminho_base = '../../data/fechamentos'


# Intervalo de datas
data_inicial = '2024-10-03'
data_final = '2024-12-31'

# Função para carregar o fechamento diário de um ativo
def carregar_ativo(nome):
    caminho = f'{caminho_base}/{nome}USDT_1d_data.csv'
    df = pd.read_csv(caminho, parse_dates=['timestamp'], index_col='timestamp')
    return df['close'].loc[data_inicial:data_final]


# Lista para armazenar resultados
resultados = []

# Loop por todos os pares possíveis
for moeda1, moeda2 in itertools.combinations(moedas, 2):
    try:
        serie1 = carregar_ativo(moeda1)
        serie2 = carregar_ativo(moeda2)

        # Alinha os índices (timestamps coincidentes)
        df_alinhado = pd.DataFrame({moeda1: serie1, moeda2: serie2}).dropna()

        if len(df_alinhado) < 30:  # ignora séries muito curtas
            continue

        pvalor, status_cointegracao = testar_cointegracao(df_alinhado[moeda1], df_alinhado[moeda2])

        resultados.append({
            'Ativo1': moeda1,
            'Ativo2': moeda2,
            'P-valor': round(pvalor, 5),
            'Cointegrado?': status_cointegracao,
            'N_observacoes': len(df_alinhado)
        })

    except Exception as e:
        print(f'Erro ao processar {moeda1} x {moeda2}: {e}')

# Converter para DataFrame e ordenar
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values('P-valor')
df_resultados.to_csv('resultados/pares_cointegrados.csv', index=False)

# Mostrar apenas os pares cointegrados (p < 0.05)
cointegrados = df_resultados[df_resultados['P-valor'] < 0.05]
print("\nPares cointegrados encontrados (p < 0.05):\n")
print(cointegrados)


Pares cointegrados encontrados (p < 0.05):

   Ativo1 Ativo2  P-valor  Cointegrado?  N_observacoes
70    LTC    TRX  0.00000          True             90
57    XRP    TRX  0.00001          True             90
19    BNB    TRX  0.00004          True             90
16    BNB    XRP  0.00102          True             90
18    BNB    LTC  0.00350          True             90
84    DOT   AVAX  0.00502          True             90
26    ETH    ADA  0.04294          True             90
